In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from qiskit_aer import Aer

# Load the dataset
iris = load_iris()
X = iris.data
y = iris.target

# Filter the dataset for Iris-setosa and Iris-virginica
mask = (y == 0) | (y == 2)
X = X[mask]
y = y[mask]

# Encode labels as -1 and 1
y = np.where(y == 0, -1, 1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Encode data using angle encoding
def encode_data(circuit, x, start_qubit):
    num_qubits = len(x)
    for i in range(num_qubits):
        circuit.ry(x[i], start_qubit + i)

# Use the quantum simulator
backend = AerSimulator()


# Design quantum circuit for inner product computation
def inner_product_circuit(x1, x2):   
    norm_data1 = np.sqrt(sum(abs(x)**2 for x in x1))
    norm_data2 = np.sqrt(sum(abs(x)**2 for x in x2))
    x1 = [x / norm_data1 for x in x1]
    x2 = [x / norm_data2 for x in x2]
    num_qubits=int(np.ceil(np.log2(len(x1))))
    qc = QuantumCircuit(2*num_qubits+1, 1)  # 4 qubits for data + 1 ancilla qubit
    qc.initialize(x1, range(0,num_qubits))  # |ψ⟩
    qc.initialize(x2, range(num_qubits,2*num_qubits))
    qc.h(2*num_qubits)
    for i in range(num_qubits):
        qc.cswap(2*num_qubits,i,num_qubits+i)
    qc.h(2*num_qubits)
    qc.measure(2*num_qubits, 0)
    return qc,norm_data1,norm_data2
def compute_inner_products(x,y):
    inner_products = np.zeros((len(x), len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            circuit,norm1,norm2 = inner_product_circuit(x[i], y[j])
            backend = Aer.get_backend('qasm_simulator')
            tqc = transpile(circuit, backend)
            shots=20000
            result = backend.run(tqc, shots=shots).result()
            # Extract the counts
            counts = result.get_counts()
            # Calculate the probabilities
            P_0 = counts.get('0', 0) / shots
            # Calculate the inner product
            inner_product = 2 * P_0 - 1
            
            inner_products[i, j] = (np.sqrt(inner_product) if inner_product>=0 else np.sqrt(-inner_product)) *norm1*norm2 
    return inner_products

def compute_classic_inner_products(x,y):
    inner_products = np.zeros((len(x), len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            inner_products[i, j] = np.inner(x[i],y[j])
    return inner_products

inner_products_train = compute_inner_products(X_train,X_train)

classic_inner_products_train=compute_classic_inner_products(X_train,X_train)
inner_products_train*=np.where(classic_inner_products_train>=0,1,-1)    

# Train the SVM
from sklearn.svm import SVC
svm = SVC(kernel='precomputed')
svm.fit(inner_products_train, y_train)

inner_products_test = compute_inner_products(X_test, X_test)
classic_inner_products_test=compute_classic_inner_products(X_test,X_test)
inner_products_test*=np.where(classic_inner_products_test>=0,1,-1)    

# Classify the test samples
y_pred = svm.predict(inner_products_test)

from sklearn.metrics import accuracy_score

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Quantum SVM Classification Accuracy: {accuracy:.2f}')

# Compare with classical baseline
svm_classical = SVC(kernel='linear')
svm_classical.fit(X_train, y_train)
y_pred_classical = svm_classical.predict(X_test)
accuracy_classical = accuracy_score(y_test, y_pred_classical)
print(f'Classical SVM Classification Accuracy: {accuracy_classical:.2f}')

